# 模型去偏
:label:`debias`

推荐系统在为用户提供个性化服务的过程中，面临着一个容易被忽视但极其重要的问题：我们用来训练模型的数据本身可靠吗？与严格控制的实验室环境不同，推荐系统的数据来源于用户在真实场景中的交互行为。这些观测数据不可避免地受到系统策略、用户习惯、物品流行度等多种因素的影响，导致数据中存在各种偏差。

更值得关注的是，推荐系统存在一个天然的反馈闭环：模型的推荐结果会影响用户的未来行为，而这些行为又会成为新的训练数据来更新模型。这种闭环机制会像滚雪球一样不断放大初始数据中的偏差，最终可能导致推荐结果的单一化和不公平性。

## 偏差的类型与影响

推荐系统中的偏差可以按照产生阶段分为两大类 :cite:`chen2023bias` ：数据偏差和结果偏差。数据偏差发生在数据收集阶段，是后续问题的根源；结果偏差则体现在推荐结果中，是数据偏差经过模型处理后的表现。

![推荐系统中的反馈回路对各种偏差的放大作用](../img/bias_in_recommendation.svg)
:width:`400px`
:label:`bias_in_recommendation`

**数据偏差**的核心问题在于训练数据的分布与理想测试数据的分布存在差异。在推荐系统中，这种差异主要体现在以下几个方面：

* **选择偏差**是显式反馈场景中的典型问题。用户倾向于只对自己感兴趣的内容进行评分，这导致我们观测到的评分数据并非所有可能评分的随机样本。研究表明，用户主动评分的物品通常获得更高的评分，而大量中性或负面的潜在评分则缺失在数据中。这种现象被称为"非随机缺失" （Missing Not At Random, MNAR） :cite:`marlin2012collaborative` ，它会使模型高估用户的整体满意度。

* **曝光偏差**则是隐式反馈场景中的核心挑战。用户只能与系统展示给他们的物品进行交互，因此一个未被观测到的交互可能有两种截然不同的含义：用户确实不感兴趣（真正的负样本），或者用户从未见过这个物品（潜在的正样本） :cite:`liu2017related` 。如果简单地将所有未观测的交互都视为负样本，模型就会学到错误的用户偏好，特别是对那些从未获得充分展示机会的长尾物品。

* **从众偏差**源于社会心理学中的群体效应。用户的行为和判断往往会受到他人意见的影响，即使这种影响与他们原本的判断相冲突。例如，当用户看到某个商品有大量好评时，他们更倾向于给出正面评价以寻求群体认同；反之，如果看到负面评价居多，也可能会随波逐流。这种从众心理使得我们收集到的反馈数据可能并非用户独立、真实的偏好表达，而是被社会舆论或群体热度所影响的结果 :cite:`krishnan2014methodology` 。

* **位置偏差**在列表推荐中尤为明显。用户通常会给予排在前面的物品更多关注，无论其实际相关性如何。这种现象源于用户的浏览习惯和对系统的信任。研究数据显示，物品的点击率会随着位置的下降而急剧衰减，这意味着"点击"这个行为不仅反映了用户偏好，还强烈地受到位置因素的影响 :cite:`collins2018study,richardson2007predicting` 。

当有偏的数据经过模型处理后，会在**推荐结果**中产生或进一步放大偏差:

* **流行度偏差**是最常见的结果偏差。由于热门物品在训练数据中占据了绝大多数的交互记录，模型会过度学习这些模式，认为"推荐热门物品总是安全的"。这导致推荐结果严重偏向热门物品，其推荐频率甚至超过了它们本身的流行度。这种现象不仅降低了推荐的个性化程度，还剥夺了长尾物品被发现的机会 :cite:`abdollahpouri2017controlling` 。

* **不公平性**则体现在系统可能对某些用户群体或物品类别产生系统性的歧视。例如，如果历史数据中某个群体的用户获得的优质推荐较少，模型就会学习并延续这种模式，在未来的推荐中继续对该群体产生不公平的对待 :cite:`wang2023survey` 。

这些偏差之间并非孤立存在，而是通过推荐系统的**反馈闭环**相互强化。热门物品由于获得更多推荐机会，会产生更多的用户交互，从而在新的训练数据中占据更大比重，进一步加剧流行度偏差。这种"富者愈富"的马太效应会持续恶化，最终可能导致推荐结果的极度单一化 :cite:`jiang2019degenerate` 。

要解决这些偏差问题，我们需要在模型设计和训练过程中引入纠偏机制。接下来我们将介绍两种经典的纠偏方法：逆倾向得分（IPS）和位置感知学习（PAL），它们分别从不同角度为我们提供了有效的解决方案。

## 用逆倾向得分（IPS）纠正选择偏差

逆倾向得分（Inverse Propensity Score, IPS） :cite:`schnabel2016recommendations` 是一种常用的纠偏方法，它的核心思想源于因果推断领域。IPS将推荐系统中的"物品展示"看作一种"干预"，通过重新加权的方式来消除数据中的选择偏差。

IPS的基本逻辑非常直观：如果一个样本本来就很容易被观测到（比如热门商品或排在首位的物品），那么它在训练中的贡献应该被适当降低；反之，如果一个样本很难被观测到（比如长尾商品或低位置的物品），但它依然被用户发现并产生了交互，那么这个样本很可能包含了更强的偏好信号，应该获得更高的权重。

在IPS框架中，**倾向得分**（Propensity Score）是关键概念，它定义为用户$u$与物品$i$的交互被观测到的概率，记作$P(O_{u,i}=1)$。这个概率反映了样本被观测到的"容易程度"。对于容易被观测到的样本（如热门物品），倾向得分较高；对于难以被观测到的样本（如长尾物品），倾向得分较低。IPS的核心操作就是用倾向得分的**倒数**作为权重，这样就实现了"逆向加权"：高倾向得分的样本获得低权重，而低倾向得分的样本获得高权重。这种设计确保了模型不会过度关注那些"容易获得"的样本，而是更加重视那些"来之不易"的样本。

**举个例子**：考虑一个电影推荐系统，其中有两类用户：恐怖片爱好者和爱情片爱好者。恐怖片爱好者很少主动观看和评分爱情片（观测概率$P=0.1$），但当他们偶尔评分时，往往给出真实的低分（如1-2分）；相反，他们会频繁观看和评分恐怖片（观测概率$P=0.8$），并给出高分（如4-5分）。如果我们用朴素方法直接统计观测数据，会发现恐怖片的平均评分远高于爱情片，这会误导模型高估两类电影之间的偏好差距。但如果使用IPS加权，那些罕见的"恐怖片爱好者给爱情片的低分评价"会获得$1/0.1=10$倍的权重，而常见的"恐怖片爱好者给恐怖片的高分评价"只获得$1/0.8=1.25$倍的权重。这样一来，模型能够更准确地估计真实的用户偏好分布，避免因观测偏差而产生的错误结论。

**IPS的数学原理**

传统的评估方法直接在观测数据上计算平均损失：

$$\hat{R}_{naive}(\hat{Y}) = \frac{1}{|D_O|} \sum_{(u,i) \in D_O} \delta_{u,i}(Y, \hat{Y})$$

其中，$D_O=\{(u,i):O_{ui}=1\}$表示观测到的数据集，$Y$表示用户对物品的真实评分，$\hat{Y}$表示模型预测的用户对物品的评分，$\delta_{u,i}$表示在$(u,i)$上损失函数（如均方误差）或者评价指标（如准确率）。研究证明，当选择偏差存在时，朴素估计器是有偏的 :cite:`steck2013evaluation` ，即$\mathbb{E}_{O}[\hat{R}_{naive}(\hat{Y})] \neq R(\hat{Y})$。

而IPS估计器则引入倾向得分加权：

$$\hat{R}_{IPS}(\hat{Y}) = \frac{1}{|U||I|} \sum_{(u,i) \in D_O} \frac{1}{P(O_{ui}=1)} \delta_{u,i}(Y, \hat{Y})$$
其中，$U$和$I$分别表示用户和物品的集合。理论上可以证明，IPS估计器是真实风险的无偏估计，即$\mathbb{E}_{O}[\hat{R}_{IPS}(\hat{Y}|P)] = R(\hat{Y})$。这个性质保证了经过IPS校正后的模型能够更准确地反映用户的真实偏好。需要注意的是，当某些样本的倾向得分非常小时，其倒数会变得异常大，可能导致估计结果不稳定。在实际应用中，通常会对权重进行适当的截断或归一化来缓解这个问题。

**应用到模型训练**

IPS不仅可以用于模型评估，还可以直接应用到模型训练中。以矩阵分解为例，传统的目标函数是：

$$\text{argmin}_{U,V} \sum_{(u,i) \in D_O} (Y_{u,i} - (u_u^T v_i + a_u + b_i + c))^2 + \lambda(||U||^2 + ||V||^2)$$
其中，$a_u$和$b_i$是用户$u$和物品$i$的偏置项，$c$是全局偏置项。

加入IPS权重后，目标函数变为：

$$\text{argmin}_{U,V} \sum_{(u,i) \in D_O} \frac{1}{P(O_{u,i}=1)} (Y_{u,i} - (u_u^T v_i + a_u + b_i + c))^2 + \lambda(||U||^2 + ||V||^2)$$

这个改动非常简洁，可以方便地集成到现有的优化算法中。

**倾向得分估计**

IPS方法的**关键挑战**在于如何准确估计倾向得分。在实际应用中，倾向得分通常是未知的，需要通过额外的模型来估计。一种简单的估计方法是**朴素贝叶斯方法**，它假设观测概率只与评分值相关，例如5分评价被观测到的概率远高于1分评价。另一种更精确的方法是**逻辑回归方法**，它利用用户特征、物品特征等丰富信息建立预测"是否被观测到"的分类模型。这种方法能够捕捉到更复杂的观测模式，通常能获得更准确的倾向得分估计。

值得注意的是，即使倾向得分的估计不够精确，IPS方法仍然能够带来显著的性能提升。研究表明，"不完美的纠偏"也远远好于"完全不纠偏"。

**如何验证纠偏效果**

理解纠偏方法的有效性需要设计合适的实验。一个经典的验证方法是**半合成实验**，它既保持了真实数据的复杂性，又允许我们控制偏差的严重程度。

**第一步：构建完整的"真实"评分矩阵**
实验从真实数据集（如MovieLens 100K）开始，这个数据集包含94.4万条评分，但评分矩阵只有6%的位置有数据。我们使用矩阵分解技术补全所有缺失的评分，得到一个完整的用户-物品评分矩阵$R_{true}$，并将这个补全的矩阵作为我们的"ground truth"。

**第二步：设计偏差模型**
接下来我们需要设计一个偏差模型来模拟真实的数据收集过程。对于评分为r的用户-物品对，我们定义其被观测到的概率：如果$r \geq 4$，观测概率为$k$（基础概率）；如果$r < 4$，观测概率为$k × \alpha^{4-r}$（递减概率）。这里参数$\alpha$控制偏差程度，当$\alpha=1$时无偏差，$\alpha$越小偏差越严重。参数$k$则调整为使总体观测率约为5%，以模拟稀疏数据的特性。

**第三步：生成有偏观测数据**
有了偏差模型，我们就能对每个用户-物品对$(u,i)$，根据其评分$R_{true}[u,i]$和上述概率模型，随机决定是否观测。这样就生成了观测矩阵$O$，其中$O[u,i]=1$表示该评分被观测到。最终得到的有偏训练数据只包含$O[u,i]=1$的评分，这就模拟了真实推荐系统中的数据收集过程。

**第四步：比较不同估计器**
在这个实验框架中，我们可以精确地比较不同估计器的效果。真实MAE是在完整的$R_{true}$矩阵上计算的真实平均绝对误差，这是我们想要估计的目标。朴素估计器直接在观测数据上计算平均误差，而IPS估计器使用$1/P(\text{观测})$作为权重计算加权平均误差。我们的比较指标是每个估计器的估计值与真实MAE之间的差异，差异越小说明估计越准确。

实验结果的显示， 当$\alpha=0.25$时，IPS估计器的误差比朴素估计器小2-3个数量级。这种量化的对比清楚地证明了纠偏方法的有效性。更重要的是，这个实验框架可以复制到任何数据集上，成为验证纠偏方法的标准做法。它不仅为研究者提供了一个客观的评估工具，也为实际应用中选择合适的纠偏方法提供了科学依据。这种实验设计的关键在于创造了一个"已知答案"的测试环境，让我们能够精确量化不同方法的效果。

IPS为推荐系统的纠偏提供了一个理论坚实且实用的框架。它的优势在于简洁性和可扩展性，可以方便地应用到各种推荐模型中。接下来我们将介绍另一种针对特定偏差类型的纠偏方法——PAL框架。

## 解耦位置偏差与用户偏好

当我们在手机上浏览推荐列表时，是否会更容易点击排在前面的内容？答案是肯定的。无论是在搜索引擎、购物应用还是视频平台，用户都有着明显的"位置偏好"——即使内容质量相同，位置靠前的物品往往获得更多点击 :cite:`richardson2007predicting` 。这种现象就是推荐系统中的位置偏差，它看似简单，实则给系统设计带来了深刻的挑战。

位置感知学习（Position-bias Aware Learning, PAL） :cite:`guo2019pal` 框架正是为了解决这一挑战而生。与IPS通过重新加权数据的思路不同，PAL采用了一种更加直接的方法：通过重新设计模型架构，将位置影响和用户真实偏好在结构层面分离开来。

PAL框架的突破来自于对用户点击行为的重新理解。研究者意识到，用户点击一个物品实际上是两个连续事件的结合：用户首先必须"看到"这个物品，然后在看到的前提下"决定点击"。这个观察包含了深刻的洞察：位置主要影响的是"看到"的概率，而不是"喜欢"的程度。

基于这个理解，PAL将点击概率进行了数学上的分解：

$$p(click|user, item, position) = p(seen|position) \times p(click|user, item, seen)$$

这个分解式的巧妙之处在于它将一个复杂的问题分解为两个相对简单的子问题。第一个子问题是纯粹的位置效应：在给定位置上，用户看到物品的概率是多少？第二个子问题是纯粹的偏好建模：当用户看到物品时，基于用户特征和物品特征，点击的概率是多少？

更重要的是，这种分解基于两个合理的假设：首先，用户看到物品的概率主要由位置决定，与物品内容关系不大；其次，当用户已经看到物品时，是否点击主要由用户偏好决定，与位置关系不大。这两个假设在直觉上是合理的，也得到了实验数据的支持。

**双模块架构**

基于上述分解，PAL设计了一个双模块的模型架构。这种设计的巧妙之处在于，它不仅实现了数学上的分解，更在架构层面强制了这种分解的逻辑。

![PAL框架与传统方法的对比](../img/pal_framework.png)
:width:`400px`
:label:`pal_framework`

ProbSeen模块专门负责建模位置效应。它的输入只有位置信息，输出是该位置被用户看到的概率。这个模块的设计相对简单，通常可以是一个浅层的神经网络，甚至可以是一个简单的查找表。它的作用是学习不同位置的"可见性"，比如第一个位置的可见性是$0.9$，第二个位置是$0.7$，第三个位置是$0.5$，等等。ProbSeen模块可以用简单的线性模型来实现，也可以用复杂的深度学习模型来实现 :cite:`guo2019pal` 。

pCTR模块则负责建模用户的真实偏好。它的输入包括用户特征、物品特征和上下文信息，但关键的是，它完全不包含位置信息。这个模块通常是一个复杂的深度学习模型，比如DeepFM :numref:`deepfm` 或者其他推荐模型。它的任务是学习用户对物品的真实偏好，而不受位置因素的干扰。

在离线训练时，两个模块的输出相乘得到最终的点击率预测：

$$bCTR = ProbSeen(position) \times pCTR(user, item, context)$$
其中$bCTR$是预测点击率。这个预测值随后与真实标签计算损失，通过反向传播同时优化两个模块的参数。

**训练与推理的分离机制**

PAL的核心技巧在于训练和推理阶段使用不同的模块组合。在训练时，两个模块联合优化，模型自动学会如何分配责任：位置靠前的物品被点击时，一部分原因归于位置效应（ProbSeen模块），另一部分归于内容质量（pCTR模块）。这种联合训练避免了我们需要事先知道每个样本真实的"看到概率"和"看到后点击概率"的困难。

在推理时，系统只使用pCTR模块进行预测。由于这个模块在训练时就被设计为不依赖位置信息，所以它能够直接给出消除位置偏差的点击率预测。这样就解决了位置信息在推理时不可用的根本问题——我们不再需要为位置特征假设一个值，pCTR模块的预测结果直接反映了用户对物品的真实偏好。

这种设计的本质是实现了信息的有效分离：位置相关的信息被ProbSeen模块处理，内容相关的信息被pCTR模块保留。推理时只使用保留的信息，从而获得更准确的偏好预测。


IPS和PAL分别代表了通用数据加权和专门结构设计两种纠偏思路，它们的成功说明了理解偏差产生机制比简单拟合数据更重要。无论技术如何演进，深入理解数据背后的偏差机制都将是构建公平、有效推荐系统的关键。